##### Copyright (C) Microsoft Corporation.  
see license file for details 


## Copies data from blob to local host


downloads to local disk data from Azure Data Storage containers (see in cell below the local and Azure storage containers structure)

In [1]:
# Local data structure:
#     /data_dir/chestxray/ChestX-ray8/  
#         BBox_List_2017.csv         
#         Data_Entry_2017.csv  
#         blacklist.csv
#         ChestXray-NIHCC/  
#             /tmp/  
#                 images_01.tar.gz    
#                 images_03.tar.gz    
#                 images_05.tar.gz   
#                 images_07.tar.gz   
#                 images_09.tar.gz   
#                 images_11.tar.gz  
#                 images_02.tar.gz   
#                 images_04.tar.gz   
#                 images_06.tar.gz   
#                 images_08.tar.gz   
#                 images_10.tar.gz   
#                 images_12.tar.gz
#             /images/
#                   00000001_000.png
#                   00000001_001.png
#                   ... 112120 png images
# 
#    Azure Data Storage container structure:
#       container nihchestxrayimages
#                   00000001_000.png
#                   00000001_001.png
#                   ... 112120 png image files, total 41.96 GiB
#       container nihchestxraydatacompressed:
#                   images_01.tar.gz
#                   ... 12 files, total 41.98 GiB
#       container nihchestxraydata
#                   BBox_List_2017.csv
#                   blacklist.csv
#                   Data_Entry_2017.csv

In [2]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext dotenv
dotenv_file_path='./../not_shared/general.env'

In [5]:
import os, platform
import sys
import urllib.request

In [6]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1063-azure-x86_64-with-debian-stretch-sid'

'/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep'

In [7]:
# import utlity functions

paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['..','src'])))]
paths_to_append
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

['/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep/../src']

[None]

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [8]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

nih_chest_xray_data_dir=os.path.join(os.path.join(*([os.sep]+prj_consts.BASE_INPUT_DIR_list)), 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))
os.makedirs(nih_chest_xray_data_dir, exist_ok=True)

nih_chest_xray_data_dir
!ls -l {nih_chest_xray_data_dir}

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC'

total 3900
drwxrwxr-x 2 loginvm022 loginvm022 3989504 Sep  4  2017 images
drwxrwxrwx 2 root       root          4096 Nov 16 05:48 tmp


In [9]:
#check azcopy is available. Uncomment second line to print _response_ if needed

response = !azcopy
response[1:10]

['azcopy 7.2.0-netcore Copyright (c) 2018 Microsoft Corp. All Rights Reserved.',
 '------------------------------------------------------------------------------',
 '# azcopy is designed for high-performance uploading, downloading, and copying',
 'data to and from Microsoft Azure Blob, and File storage.',
 '',
 '# Command Line Usage:',
 '    azcopy --source <source> --destination <destination> [options]',
 '',
 '# Options:']

In [10]:
%dotenv $dotenv_file_path

storage_account_key = os.getenv('sa_key')

###### Copy meta-data  
 - NIH Chest Xray: BBox_List_2017.csv, Data_Entry_2017.csv  
 - Azure Chest Xray: blacklist.csv   

In [11]:
# copy data to local host dir
source_container = 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name')
dest_local_dir = os.path.join(nih_chest_xray_data_dir, '..')

!azcopy --quiet \
    --source {source_container}  \
    --destination {dest_local_dir} \
    --source-key {storage_account_key} \
    --verbose \
    --include "BBox_List_2017.csv"

!azcopy --quiet \
    --source {source_container}  \
    --destination {dest_local_dir} \
    --source-key {storage_account_key} \
    --verbose \
    --include "Data_Entry_2017.csv"

!azcopy --quiet \
    --source {source_container}  \
    --destination {dest_local_dir} \
    --source-key {storage_account_key} \
    --verbose \
    --include "blacklist.csv"

=[2019/11/17 22:16:33.224+00:00] >>>>>>>>>>>>>>>>
[2019/11/17 22:16:33.232+00:00][VERBOSE] Finished: 0 file(s), 0 B; Average Speed:0 B/s.
[2019/11/17 22:16:33.236+00:00][VERBOSE] 7.2.0 : azcopy --quiet --source https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata --destination /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/.. --source-key ****** --verbose --include BBox_List_2017.csv
[2019/11/17 22:16:33.310+00:00][VERBOSE] Attempt to parse address 'https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata' to a directory as a candidate location succeeded.
[2019/11/17 22:16:33.311+00:00][VERBOSE] Source is interpreted as a Cloud blob directory: https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata.
[2019/11/17 22:16:33.313+00:00][VERBOSE] Attempt to parse address '/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/..' to a directory as a candidate location succeeded.
[2019/11/17 22:16:33.315+00:00][VERBOSE] Destination is interpreted as a Loc

###### Copy NIH Chest Xray  compressed image data
Make:
```
copy_NIH_compressed_data_FLAG = False
``` 
to skip next 2 cells and use the following cells to download uncompressed images.

Make:
```
copy_NIH_compressed_data_FLAG = True
```
if you want to download compressed data and uncompress locally in the next 2 cells. 

In [12]:
copy_NIH_compressed_data_FLAG = False

if copy_NIH_compressed_data_FLAG:
    tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
    tmp_dir
    os.makedirs(tmp_dir, exist_ok=True)

    compressed_data_container = 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name_compressed_data')

    answer = !azcopy --quiet \
    --source {compressed_data_container}  \
    --destination {tmp_dir} \
    --source-key {crt_key} \
    --recursive \
    --verbose \
    --include "images_"
    
    print(answer[-7:])
    

The azcopy download process above will run for several minutes. 

Make sure you have these file sizes when it finishes:  
total __44023372__  
-rw-r--r-- 1 1000 users __2008470987__ Sep 26 17:19 images_001.tar.gz  
-rw-r--r-- 1 1000 users __3952623504__ Sep 26 19:05 images_002.tar.gz  
-rw-r--r-- 1 1000 users __3929234850__ Sep 26 17:22 images_003.tar.gz  
-rw-r--r-- 1 1000 users __3838903983__ Sep 26 17:25 images_004.tar.gz  
-rw-r--r-- 1 1000 users __3935496531__ Sep 26 17:22 images_005.tar.gz  
-rw-r--r-- 1 1000 users __3986301172__ Sep 26 17:25 images_006.tar.gz  
-rw-r--r-- 1 1000 users __4016328426__ Sep 26 19:05 images_007.tar.gz  
-rw-r--r-- 1 1000 users __4018347353__ Sep 26 19:05 images_008.tar.gz  
-rw-r--r-- 1 1000 users __4111327929__ Sep 26 19:05 images_009.tar.gz  
-rw-r--r-- 1 1000 users __4181556296__ Sep 26 19:05 images_010.tar.gz  
-rw-r--r-- 1 1000 users __4187084020__ Sep 26 19:05 images_011.tar.gz  
-rw-r--r-- 1 1000 users __2914187733__ Sep 26 17:21 images_012.tar.gz  
  
If the unzipping process below does not end up with __112120__ .png images, you may need to repeat the download process for the above 12 .tar.gz files.

The uncompressiing process below will run for several minutes.

While it runs, you can monitor the progress by ssh-ing 
into your compute context machine and running:   
find /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/images  -type f | wc -l  

The final number of images is:  
__112120__  



In [13]:
if copy_NIH_compressed_data_FLAG:
    print(nih_chest_xray_data_dir)
    print(tmp_dir)
    !ls -l $tmp_dir
    # !find $nih_chest_xray_data_dir -type f | wc -l

    old_crt_dir = os.getcwd()
    os.chdir(nih_chest_xray_data_dir)
    !pwd    
    !ls -l $tmp_dir/*.tar.gz 
    allImages = ! cat $tmp_dir/*.tar.gz | tar -zxf - -i
    # !for file in $tmp_dir/*.tar.gz; do tar -zxf $file; done
    os.chdir(old_crt_dir)
    !pwd

##### Download uncomressed data (112120 .png images like 00000001_000.png) to local host dir

In [14]:
if not copy_NIH_compressed_data_FLAG:
    source_container = 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name_images')
    dest_local_dir = os.path.join(nih_chest_xray_data_dir, 'images')

    !azcopy --quiet \
        --source {source_container}  \
        --destination {dest_local_dir} \
        --source-key {storage_account_key} \
        --recursive

=Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 444.867 KB; Average Speed:47.56 KB/s.                     Finished: 1957 file(s), 756.698 MB; Average Speed:66.44 MB/s.                  Finished: 3972 file(s), 1.493 GB; Average Speed:113.75 MB/s.                   Finished: 6003 file(s), 2.244 GB; Average Speed:148.34 MB/s.                   Finished: 8048 file(s), 2.99 GB; Average Speed:174.56 MB/s.                    Finished: 10244 file(s), 3.801 GB; Average Speed:198.74 MB/s.                  Finished: 12113 file(s), 4.492 GB; Average Speed:212.66 MB/s.                  Finished: 14293 file(s), 5.288 GB; Average Speed:228.53 MB/s.                  Finished: 16440 file(s), 6.073 GB; Average Speed:2

Finished: 67086 file(s), 24.675 GB; Average Speed:139.31 MB/s.                 Finished: 67529 file(s), 24.843 GB; Average Speed:138.69 MB/s.                 Finished: 67924 file(s), 24.989 GB; Average Speed:137.97 MB/s.                 Finished: 68269 file(s), 25.119 GB; Average Speed:137.17 MB/s.                 Finished: 68623 file(s), 25.252 GB; Average Speed:136.41 MB/s.                 Finished: 69027 file(s), 25.403 GB; Average Speed:135.76 MB/s.                 Finished: 69415 file(s), 25.551 GB; Average Speed:135.1 MB/s.                  Finished: 69894 file(s), 25.729 GB; Average Speed:134.62 MB/s.                 Finished: 70054 file(s), 25.787 GB; Average Speed:133.53 MB/s.                 Finished: 70658 file(s), 26.016 GB; Average Speed:133.33 MB/s.                 Finished: 71034 file(s), 26.152 GB; Average Speed:132.67 MB/s.                 Finished: 71576 file(s), 26.356 GB; Average Speed:132.36 MB/s.                 Finished: 71976 file(s), 26.507 GB; Average Speed:13

Finished: 101349 file(s), 37.838 GB; Average Speed:106.55 MB/s.                Finished: 101771 file(s), 38.001 GB; Average Speed:106.41 MB/s.                Finished: 102190 file(s), 38.164 GB; Average Speed:106.27 MB/s.                Finished: 102647 file(s), 38.342 GB; Average Speed:106.18 MB/s.                Finished: 103055 file(s), 38.497 GB; Average Speed:106.02 MB/s.                Finished: 103536 file(s), 38.683 GB; Average Speed:105.94 MB/s.                Finished: 104058 file(s), 38.885 GB; Average Speed:105.92 MB/s.                Finished: 104556 file(s), 39.079 GB; Average Speed:105.87 MB/s.                Finished: 105013 file(s), 39.256 GB; Average Speed:105.78 MB/s.                Finished: 105478 file(s), 39.433 GB; Average Speed:105.68 MB/s.                Finished: 105829 file(s), 39.566 GB; Average Speed:105.47 MB/s.                Finished: 106131 file(s), 39.68 GB; Average Speed:105.22 MB/s.                 Finished: 106460 file(s), 39.805 GB; Average Speed:1

In [15]:
 
!jupyter nbconvert --to html 001_get_data_B.ipynb

[NbConvertApp] Converting notebook 001_get_data_B.ipynb to html
[NbConvertApp] Writing 286135 bytes to 001_get_data_B.html
